### Keras Tuner con el problema MNIST

Keras Tuner es una librería bastante reciente que simplifica en gran medida el ajuste de los hiperparámetros de una red neuronal. Toda la documentación en este enlace:

https://keras-team.github.io/keras-tuner/

In [1]:
!pip install -q -U keras-tuner
import kerastuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.4 MB/s eta 0:00:00


<ipython-input-1-5d7b6944a6ac>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


Carga de los datos:

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
from time import time
import shutil

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Normalización:

In [4]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

Lo primero que hay que hacer es definir un hipermodelo, que es una función que genera un modelo de Keras que depende de unos hiperparámetros con los que vamos a jugar. Los hiperparámetros se muestrean a partir del argumento ``hp`` de la función.

En este ejemplo sólo vamos a ajustar la constante de regularización de la capa oculta:

In [5]:
def model_builder(hp):
  hp_lambda = hp.Choice('lambda', values = [1.0, 0.1, 0.01, 0.001, 0.0001]) 
  hp_lr = hp.Choice('lr', values = [1.0, 0.1, 0.01, 0.001, 0.0001]) 
  
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units = 50, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_lambda)))
  model.add(keras.layers.Dense(10, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_lr), 
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])    
  
  return model

Borramos el directorio de logs:

In [6]:
!rm -rf my_dir/intro_to_kt/

Lo siguiente es crear un ``tuner`` para hacer el ajuste de los hiperparámetros. Existen distintos tipos:

- RandomSearch
- Hyperband
- BayesianOptimization
- Sklearn

Lo más fácil es hacer una búsqueda aleatoria con ``RandomSearch``. Al crear el ``tuner`` hay que especificar:

- El hipermodelo.
- La variable a optimizar.
- El número total de pruebas.
- El número de ejecuciones por prueba.

In [7]:
tuner = kt.RandomSearch(model_builder, 
                        objective='val_acc',
                        max_trials=10,
                        executions_per_trial=3,
                        directory='my_dir',
                        project_name='intro_to_kt')

Un resumen del espacio de búsqueda:

In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
lambda (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}
lr (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}


Y lanzamos la búsqueda:

In [9]:
tuner.search(img_train, label_train,
             epochs=1,
             validation_data=(img_test, label_test))

Trial 10 Complete [00h 00m 18s]
val_acc: 0.9260999957720438

Best val_acc So Far: 0.9260999957720438
Total elapsed time: 00h 03m 32s


Acceso al mejor modelo. Hay que tener en cuenta que ya está entrenado, y siempre es mejor reentrenarlo con todos los datos. 

In [10]:
best_model = tuner.get_best_models()[0]
best_model.evaluate(img_test, label_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.4141 - acc: 0.9298


[0.41412270069122314, 0.9297999739646912]

Resumen de los resultados:

In [11]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_acc", direction="max")

Trial 09 summary
Hyperparameters:
lambda: 0.001
lr: 0.01
Score: 0.9260999957720438

Trial 02 summary
Hyperparameters:
lambda: 0.0001
lr: 0.0001
Score: 0.9020333488782247

Trial 08 summary
Hyperparameters:
lambda: 0.01
lr: 0.01
Score: 0.8801999886830648

Trial 04 summary
Hyperparameters:
lambda: 0.1
lr: 0.001
Score: 0.8776000142097473

Trial 03 summary
Hyperparameters:
lambda: 0.1
lr: 0.0001
Score: 0.8741666475931803

Trial 01 summary
Hyperparameters:
lambda: 0.001
lr: 0.1
Score: 0.5332666635513306

Trial 00 summary
Hyperparameters:
lambda: 0.0001
lr: 0.1
Score: 0.4631333351135254

Trial 05 summary
Hyperparameters:
lambda: 0.0001
lr: 1.0
Score: 0.11006666719913483

Trial 07 summary
Hyperparameters:
lambda: 0.1
lr: 1.0
Score: 0.10576666643222173

Trial 06 summary
Hyperparameters:
lambda: 1.0
lr: 0.1
Score: 0.10073333481947581


Obtenemos los parámetros del mejor modelo y lo reentrenamos:

In [12]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4903 - acc: 0.9059 - val_loss: 0.4436 - val_acc: 0.9147
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4510 - acc: 0.9211 - val_loss: 0.4068 - val_acc: 0.9353
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4384 - acc: 0.9214 - val_loss: 0.4170 - val_acc: 0.9258
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4210 - acc: 0.9232 - val_loss: 0.5134 - val_acc: 0.8873
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4226 - acc: 0.9231 - val_loss: 0.5140 - val_acc: 0.8912
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4183 - acc: 0.9237 - val_loss: 0.3833 - val_acc: 0.9312
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4143 - acc: 0.9238 - val_loss: 0.4671 - val_acc: 0.9003
Epoch 8/10
1875/1875 [==============================] - 4s 2ms

#### Otro ejemplo: 

Ajuste del learning rate, el parámetro de regularización y el número de neuronas en la capa oculta con un tuner de tipo hyperband.

https://arxiv.org/pdf/1603.06560.pdf

Hipermodelo:

In [13]:
def model_builder_2(hp):
  hp_lambda = hp.Choice('lambda', values = [0.001, 0.0001]) 
  hp_units = hp.Int('units', min_value = 32, max_value = 128, step = 32)
  hp_learning_rate = hp.Choice('learning_rate', values = [1.0, 0.1, 0.01, 0.001]) 
  
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_lambda)))
  model.add(keras.layers.Dense(10, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])    
  
  return model

Borramos la carpeta de logs:

In [14]:
!rm -rf my_dir/intro_hyperband/

Creamos el tuner:

In [15]:
tuner = kt.Hyperband(model_builder_2,
                     objective = 'val_acc', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_hyperband')  

Resumen del espacio de búsqueda:

In [16]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
lambda (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001], 'ordered': True}


Búsqueda:

In [17]:
tuner.search(img_train, label_train,
             epochs=10,
             validation_data=(img_test, label_test))

Trial 30 Complete [00h 01m 15s]
val_acc: 0.9496999979019165

Best val_acc So Far: 0.9783999919891357
Total elapsed time: 00h 15m 03s


Mejores hiperparámetros:

In [18]:
best_hps = tuner.get_best_hyperparameters()[0]
best_hps.values

{'lambda': 0.0001,
 'units': 96,
 'learning_rate': 0.001,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0021'}

Reentrenamiento del modelo:

In [19]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2929 - acc: 0.9225 - val_loss: 0.1708 - val_acc: 0.9552
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1551 - acc: 0.9615 - val_loss: 0.1350 - val_acc: 0.9671
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1239 - acc: 0.9716 - val_loss: 0.1336 - val_acc: 0.9689
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1075 - acc: 0.9769 - val_loss: 0.1236 - val_acc: 0.9717
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0978 - acc: 0.9802 - val_loss: 0.1276 - val_acc: 0.9702
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0912 - acc: 0.9818 - val_loss: 0.1126 - val_acc: 0.9744
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0851 - acc: 0.9840 - val_loss: 0.1152 - val_acc: 0.9737
Epoch 8/10
1875/1875 [==============================] - 6s 3ms